In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, f1_score
from sklearn.metrics import mean_squared_error

In [2]:
# Генерируем уникальный seed
my_code = "Игнатьева"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [4]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [12]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [13]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.436364,0.475728,0.483636,0.537162,0.849276,0.770044,Bream
1,0.545455,0.563107,0.556364,0.550676,0.545806,0.984362,Perch
2,0.666667,0.61165,0.610909,0.60473,0.642425,0.888916,Perch
3,0.007394,0.08932,0.083636,0.084459,0.031258,0.031795,Smelt
4,0.181818,0.469903,0.465455,0.489865,0.230512,0.475001,Pike
...,...,...,...,...,...,...,...
90,0.121212,0.266019,0.265455,0.287162,0.499878,0.399499,Parkki
91,0.047273,0.194175,0.189091,0.209459,0.222834,0.283559,Roach
92,0.181818,0.530097,0.525455,0.523649,0.264232,0.453896,Pike
93,0.415152,0.514563,0.510909,0.510135,0.530961,0.888244,Perch


In [14]:
# Задание №1 - анализ метода опорных векторов в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [15]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Length1', 'Length2', 'Length3']
Width


In [16]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [17]:
y_train


0     0.770044
1     0.984362
2     0.888916
3     0.031795
4     0.475001
        ...   
90    0.399499
91    0.283559
92    0.453896
93    0.888244
94    0.538911
Name: Width, Length: 95, dtype: object

In [18]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
r_model1 = SVR( kernel='linear')
r_model2 = SVR(kernel='poly')
r_model3 = SVR()
r_model4 = SVR(kernel='sigmoid')




In [19]:
r_model1.fit(x_train,y_train)

a=r_model1.predict(x_test)
mse1 = mean_squared_error(y_test, a)
mse1

0.014520212411616884

In [20]:

r_model2.fit(x_train,y_train)
a=r_model2.predict(x_test)
mse2 = mean_squared_error(y_test, a)
mse2

0.04385556174598257

In [21]:
r_model3.fit(x_train,y_train)
a=r_model3.predict(x_test)
mse3 = mean_squared_error(y_test, a)
mse3

0.014455873241371727

In [22]:
r_model4.fit(x_train,y_train)
a=r_model4.predict(x_test)
mse4 = mean_squared_error(y_test, a)
mse4

26.98823884003972

In [23]:
# Задание №2 - анализ метода опорных векторов в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [24]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Height', 'Width']
Species


In [25]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [26]:
y_train

0      Bream
1      Perch
2      Perch
3      Smelt
4       Pike
       ...  
90    Parkki
91     Roach
92      Pike
93     Perch
94      Pike
Name: Species, Length: 95, dtype: object

In [27]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
c_model1 = SVC(kernel= 'linear',degree=1)
c_model2 = SVC(kernel='poly',degree=0.5)
c_model3 = SVC(degree=0)
c_model4 = SVC(kernel='sigmoid',degree=1)


In [28]:
c_model1.fit(x_train, y_train)
print(c_model1.predict(x_test))

['Perch' 'Perch' 'Perch' 'Bream' 'Perch' 'Perch' 'Bream' 'Perch' 'Perch'
 'Perch' 'Bream' 'Bream' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Bream' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Bream']


In [29]:
c_model2.fit(x_train, y_train)
print(c_model2.predict(x_test))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch']


In [30]:
c_model3.fit(x_train, y_train)
print(c_model3.predict(x_test))

['Perch' 'Perch' 'Perch' 'Bream' 'Perch' 'Perch' 'Bream' 'Smelt' 'Perch'
 'Perch' 'Bream' 'Bream' 'Perch' 'Parkki' 'Perch' 'Perch' 'Perch' 'Perch'
 'Bream' 'Smelt' 'Parkki' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Bream']


In [31]:
c_model4.fit(x_train, y_train)
print(c_model4.predict(x_test))

['Bream' 'Bream' 'Bream' 'Perch' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream'
 'Perch' 'Perch' 'Perch' 'Bream' 'Bream' 'Bream' 'Bream' 'Perch' 'Bream'
 'Bream' 'Smelt' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream'
 'Bream' 'Bream' 'Bream' 'Bream' 'Bream']
